In [1]:
from qm1.grid import *
from qm1.qmsystem import *  
# set up a grid (with vanishing boundary conditions)
grid = UniformGrid(boundary_condition="periodic", xmin=-5., xmax=5., num=50)
# define potential to use
stat_pot = BarrierPot(xstart=-2., xstop=+2., vstep=-1.)
# set the quantum mechanical system
qsys = QMSystem(stat_pot=stat_pot, grid=grid)
# set time domain
tgrid=np.linspace(0., 20., 1000)

In [6]:
from qm1.operators import *
# construct some operators, that act on the wave function
op_identity = IdentityOp(qsys.grid)
op_position = PositionOp(qsys.grid)
op_momentum = MomentumOp(qsys.grid)
op_kinetic = KineticOp(qsys)
op_stat_pot = StatPotentialOp(qsys)
# make the operators more efficient
make_efficient([op_identity, op_kinetic, op_position, op_momentum, op_stat_pot])
# time dependent potential operator (local) - perturbation
op_td_pot = OperatorTD(qsys.grid, DipolTDPot())
op_td_pot = OperatorTD(qsys.grid, ZeroTDPot())
# potential and hamilton operator
op_potential = OperatorTD(qsys.grid)
op_potential = op_td_pot + op_stat_pot
op_hamilton = op_potential + op_kinetic
# hamiltonian at time 0
op_hamilton_gs = op_hamilton.eval(t=tgrid[0])
print(type(op_hamilton_gs))
# show a few operators
# return the rhs of the schrödinger equation, when lhs is only the time derivative, rhs = i hbar H
op_rhs = OperatorTD(qsys.grid)
print(type(op_hamilton_gs))
op_rhs += op_hamilton_gs * (-1j)

op_td_pot.show(tgrid=tgrid[::10], file='op_td_pot.gif')
op_stat_pot.show(file='op_stat_pot.png')
op_hamilton.show(tgrid=tgrid[::10], file='op_hamilton.gif')

<class 'qm1.operators.OperatorConst'>
<class 'qm1.operators.OperatorConst'>


In [3]:
from qm1.eigensystem import Eigensystem
import pandas
eigsys = Eigensystem(qsys=qsys, operator=op_hamilton_gs)
eigsys.show('eigensystem.png')
obs = eigsys.get_observables([op_identity, op_position, op_momentum, op_hamilton_gs])
pandas.DataFrame(np.real(obs[:,:,0]), columns=['op_identity','op_position', 'op_momentum', 'op_hamilton_gs'],  index=['state '+str(_is) for _is in range(eigsys.num)])

,op_identity,op_position,op_momentum,op_hamilton_gs
state 0,1.0,-1.242127e-15,0.0,-0.835221
state 1,1.0,7.160180e-16,0.0,-0.378701
state 2,1.0,1.088019e-15,0.0,0.043500
state 3,1.0,1.372236e-14,0.0,0.410592
state 4,1.0,-8.969908e-15,0.0,0.483808
state 5,1.0,6.528111e-15,0.0,1.358302
state 6,1.0,1.966483e-15,0.0,1.416733
state 7,1.0,-1.708356e-15,0.0,2.669492
state 8,1.0,1.353584e-13,0.0,2.749653
state 9,1.0,-1.439231e-11,0.0,4.361140


In [4]:
from qm1.wavefunction import GaussianWavePackage
init_wf = GaussianWavePackage(qsys.grid, mu=0, sigma=1, k=0.1)
init_wf.show('init_wavefunc.png')
obs = init_wf.get_observables([op_identity, op_position, op_momentum, op_hamilton_gs])
pandas.DataFrame(np.real(obs.transpose()), columns=['op_identity', 'op_position', 'op_momentum', 'op_hamilton_gs'],  index=['exp-val', 'variance'])

,op_identity,op_position,op_momentum,op_hamilton_gs
exp-val,1.0,-1.372560e-17,-0.098998,-0.741754
variance,0.0,5.000000e-01,0.489938,0.111615


In [7]:
psis, tgrid = init_wf.evolve(tgrid, op_rhs)

In [8]:
def print_evolution(psis, tgrid, file):
  import matplotlib.pyplot as plt
  from matplotlib.animation import FuncAnimation
  fig, ax = plt.subplots(figsize=(10, 10))
  ymin = min([np.min(np.abs(_psi.func)**2) for _psi in psis])
  ymax = max([np.max(np.abs(_psi.func)**2) for _psi in psis])

  def animate(i):
    ax.clear()
    ax.set_title('evolution of wavefunction')
    ax.set_xlabel('position')
    ax.set_ylabel('density of the wave function')
    ax.set_ylim((ymin, ymax))
    line = ax.plot(psis[0].grid.points, np.abs(psis[i].func)**2)
    text = ax.text(0.8, 0.9, 'time='+str(tgrid[i]), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    return [line, text]
  ani = FuncAnimation(fig=fig, func=animate, frames=range(len(psis)), interval=1000./24.)
  ani.save(file, writer='imagemagick', fps=24)
  plt.close()

print_evolution(psis[::10], tgrid, file='wavefunc.gif')

In [ ]:
# Display the animation in the jupyter notebook
# HTML(anim.to_jshtml())